### Use case notebook for the cadence python package ###

In [1]:
import os
import glob
import random
import numpy as np

# There is a warning in the timeboard library that we want to suppress here
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays
from faker import Faker
import operator
from functools import reduce
from itertools import chain

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.mscheduler import Meetings
print(f'Package version: {cadence.__version__}')

Package version: 0.0.post1.dev21+ga7607f0.d20240825


### Helper functions ###

In [2]:
# We randomly assign students to groups
def make_random_groups(students, number_of_groups):
    #Shuffle list of students
    random.seed(seed)
    random.shuffle(students)
    #Create groups
    all_groups = []
    for index in range(number_of_groups):
        group = students[index::number_of_groups]
        all_groups.append(group)
    return all_groups

# Data directory
data_dir = os.path.join(os.environ.get('HOME'), 'data')
print(f'Data will be saved in "{data_dir}"')

Data will be saved in "/app/data"


### Create the data: meeting participants and research groups ###

In [3]:
# Let's make this reproducible
seed = 242
fake = Faker()
Faker.seed(seed)

# Create a group of meeting participants
n_participants = 10
n_groups = 3
name_list = [fake.name().split(' ')[0] for n in range(n_participants)]

# Tranform into a data frame
name_df = pd.DataFrame({'name': name_list})

# Assign the names to groups
name_df = name_df.assign(group=None)
group_list = make_random_groups(students=list(name_df.get('name').unique()), number_of_groups=n_groups)

for group in range(len(group_list)):
    name_df.loc[name_df['name'].isin(group_list[group]), 'group'] = f'group_{group}'

name_df = name_df.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)
display(name_df)

,name,group
0,Aaron,group_0
1,Erica,group_0
2,Kayla,group_0
3,Virginia,group_0
4,Darren,group_1
5,David,group_1
6,Jamie,group_1
7,Brandi,group_2
8,Geoffrey,group_2
9,Jon,group_2


### Instantiate the Meeting class ###

In [4]:
groups = list(name_df['group'].values)
names = list(name_df['name'].values)
meet = Meetings(name_list=names, group_list=groups)
display(meet.name_df)

,name,group
0,Aaron,group_0
1,Erica,group_0
2,Kayla,group_0
3,Virginia,group_0
4,Darren,group_1
5,David,group_1
6,Jamie,group_1
7,Brandi,group_2
8,Geoffrey,group_2
9,Jon,group_2


### Create a presenter list with a random sequence of names from alternating groups ###
This is because we may want a member of a different group present every week

In [5]:
# Instantiate the Meetings instance with the original data
groups = list(name_df['group'].values)
names = list(name_df['name'].values)
meet = Meetings(name_list=names, group_list=groups)

# Create a presenter list with the names from alternating groups
presenter_list = meet.create_name_sequence()
print(presenter_list)
print()
# This changes the sequence of names in the data frame of the instance
display(meet.name_df)

['Aaron', 'Darren', 'Brandi', 'Erica', 'David', 'Geoffrey', 'Kayla', 'Jamie', 'Jon', 'Virginia']



,name,group
0,Aaron,group_0
1,Darren,group_1
2,Brandi,group_2
3,Erica,group_0
4,David,group_1
5,Geoffrey,group_2
6,Kayla,group_0
7,Jamie,group_1
8,Jon,group_2
9,Virginia,group_0


### Manipulate the sequence of presenters ###

In [6]:
# Instantiate the Meetings instance with the original data
groups = list(name_df['group'].values)
names = list(name_df['name'].values)
meet = Meetings(name_list=names, group_list=groups)

# This is the list of names that we generated
print(f'Input list of names:\n{names}')

# Create an initial list of presenters from alternating groups
presenter_list_0 = meet.create_name_sequence()
print()
print(f'Presenter list after merging groups:\n{presenter_list_0}')
display(meet.name_df)

Input list of names:
['Aaron', 'Erica', 'Kayla', 'Virginia', 'Darren', 'David', 'Jamie', 'Brandi', 'Geoffrey', 'Jon']

Presenter list after merging groups:
['Aaron', 'Darren', 'Brandi', 'Erica', 'David', 'Geoffrey', 'Kayla', 'Jamie', 'Jon', 'Virginia']


,name,group
0,Aaron,group_0
1,Darren,group_1
2,Brandi,group_2
3,Erica,group_0
4,David,group_1
5,Geoffrey,group_2
6,Kayla,group_0
7,Jamie,group_1
8,Jon,group_2
9,Virginia,group_0


In [7]:
# Let's drop a couple of names. Some names that we do not want on the schedule
np.random.seed(seed)
drop_names = list(np.random.choice(presenter_list_0, size=2, replace=False))
print()
print(f'Dropping names: {drop_names}')
presenter_list_1 = [name for name in presenter_list_0 if name not in drop_names]
print(f'Presenter list after dropping the names:\n{presenter_list_1}')

# Still, we want to switch positions 1 and 4 in the sequence
presenter_list_2 = presenter_list_1.copy()
presenter_list_2[1], presenter_list_2[4] = presenter_list_2[4], presenter_list_2[1]
print()
print(f'Presenter list after switching positions 1 and 4:\n{presenter_list_2}')


Dropping names: ['Jon', 'Virginia']
Presenter list after dropping the names:
['Aaron', 'Darren', 'Brandi', 'Erica', 'David', 'Geoffrey', 'Kayla', 'Jamie']

Presenter list after switching positions 1 and 4:
['Aaron', 'David', 'Brandi', 'Erica', 'Darren', 'Geoffrey', 'Kayla', 'Jamie']


In [8]:
# Now, we are happy with that sequence and we want use it in the meeting schedule
presenter_list_3 = meet.create_name_sequence(name_sequence=presenter_list_2, merge_groups=False)
print()
print(f'Presenter sequence for meeting schedule:{presenter_list_3}')
print('Group distribution')
display(meet.name_df)


Presenter sequence for meeting schedule:['Aaron', 'David', 'Brandi', 'Erica', 'Darren', 'Geoffrey', 'Kayla', 'Jamie']
Group distribution


,name,group
0,Aaron,group_0
1,David,group_1
2,Brandi,group_2
3,Erica,group_0
4,Darren,group_1
5,Geoffrey,group_2
6,Kayla,group_0
7,Jamie,group_1


### Create the meeting schedule ###

In [9]:
# Create a meeting schedule for three months
start_date = '2024-08-07'
end_date = '2024-10-31'
# pick a start name
seed = 235
start_name = np.random.choice(meet.name_df['name'].unique(), size=1)[0]
print(start_name)
print()
cal = meet.create_timeboard(start_date=start_date,
                            end_date=end_date,
                            start_name=start_name)
display(cal)

Darren



,date,name,group,holiday,comment
0,2024-08-07,Darren,group_1,False,None
1,2024-08-14,Geoffrey,group_2,False,None
2,2024-08-21,Kayla,group_0,False,None
3,2024-08-28,Jamie,group_1,False,None
4,2024-09-04,Aaron,group_0,False,None
5,2024-09-11,David,group_1,False,None
6,2024-09-18,Brandi,group_2,False,None
7,2024-09-25,Erica,group_0,False,None
8,2024-10-02,Darren,group_1,False,None
9,2024-10-09,Geoffrey,group_2,False,None


### Skip meeting dates with a comment ###

In [10]:
# We want to skip a date where we are going to have a party
party_date = '2024-09-18'

comment = 'Department party, no meeting'
name = 'Everyone'

cal_skipped = meet.skip_date(cal_df=cal, date=party_date, comment=comment, name='Everyone')

# As you can see, the skipped date is marked and the sequence continues after the skipped date
display(cal_skipped)

,date,name,group,holiday,comment
0,2024-08-07,Darren,group_1,False,None
1,2024-08-14,Geoffrey,group_2,False,None
2,2024-08-21,Kayla,group_0,False,None
3,2024-08-28,Jamie,group_1,False,None
4,2024-09-04,Aaron,group_0,False,None
5,2024-09-11,David,group_1,False,None
6,2024-09-18,Everyone,group_2,False,"Department party, no meeting"
7,2024-09-25,Brandi,group_2,False,None
8,2024-10-02,Erica,group_0,False,None
9,2024-10-09,Darren,group_1,False,None


In [11]:
# We cannot skip a date that is not on the list
# party_date = '2024-09-20'
# cal_skipped = meet.skip_date(cal_df=cal, date=party_date, comment=comment, name='Everyone')

### Swap two dates in meeting schedule ###

In [12]:
date_1 = '2024-08-21'
date_2 = '2024-10-02'

cal_swapped = meet.swap_dates(cal_df=cal_skipped, date_1=date_1, date_2=date_2)

display(cal_swapped)

,date,name,group,holiday,comment
0,2024-08-07,Darren,group_1,False,None
1,2024-08-14,Geoffrey,group_2,False,None
2,2024-08-21,Erica,group_0,False,None
3,2024-08-28,Jamie,group_1,False,None
4,2024-09-04,Aaron,group_0,False,None
5,2024-09-11,David,group_1,False,None
6,2024-09-18,Everyone,group_2,False,"Department party, no meeting"
7,2024-09-25,Brandi,group_2,False,None
8,2024-10-02,Kayla,group_0,False,None
9,2024-10-09,Darren,group_1,False,None


In [14]:
# Save the data
data_file = 'meetings.csv'
cal_swapped.to_csv(os.path.join(data_dir, data_file), index=False)